We will define here the MassSample class, which will allow us ti characterize each sample by its position and mass.

In [ ]:
import numpy as np
import numpy.random as rd
import scipy.stats as st

import numpy.typing as npt

mu = 1.0
size = 10.0

class MassSample:
    """
    Fields:
      mass: mass of the particle (float)
      position: position of the particle (vec3)
      previous_position: previous position of the particle (for Verlet integration) (vec3)
    Methods:
      speed: speed of the particle (float -> vec3)
    """
    def __init__(self):
        self.mass: float = mu
        self.pos: npt.NDArray = rd.rand(3) * size
        self.prev_pos: npt.NDArray = np.copy(self.pos) ## + rd.randn(3)
        self.index: int = 0

    def speed(self, dt:float) -> npt.NDArray:
        return (self.pos - self.prev_pos) / dt

    def update_id(self,index:int):
        self.index = index
        return
    
test_sample = MassSample()

print(test_sample.pos)
print(test_sample.prev_pos)
print(test_sample.speed(1))



We will define here the FMMCell class, allowing us to represent the MassSamples contained in a cell and its characteristics, such as its width, samples, barycenter, mass, field tensor and neighbors

In [ ]:
from typing import List

class FMMCell:
    """
    Fields:
       width: max distance between barycenter and contained particles (float)
       samples: list of mass samples in the cell, None if not leaf (List[MassSample] | None)
       barycenter: centre of mass (vec3)
       mass: mass of the cell (float)
       field_tensor: when leaf cell sum of field tensor contributions of all other far cells (vec4)
       neighbors: list of neighboring cells (List[FMMCell])

    """
    def __init__(self, samples, centroid, size):
        self.samples: npt.Optional[npt.List[MassSample]] = samples
        self.mass: float = 0
        self.barycenter: npt.NDArray = 0
        self.width: float = 0
        self.centroid: npt.NDArray = centroid
        self.size: float = size
        for s in samples:
            self.mass += s.mass
            self.barycenter += s.mass * s.pos
        self.barycenter /= self.mass
        for s in samples:    
            if (np.linalg.norm(s.pos-self.barycenter)>self.width):
                self.width = np.linalg.norm(s.pos)
        self.neighbors: List[int] = []
        self.field_tensor: npt.NDArray = np.zeros(4)

    def contains_sample(self,sample:MassSample)->bool:
        print(np.max(abs(self.centroid - sample.pos)))
        print(self.size/2)
        return (np.max(abs(self.centroid - sample.pos)) < self.size/2)


test_sample = MassSample()
test_cell = FMMCell([test_sample],[5,5,5],5)


print(test_sample.pos)
print(test_cell.contains_sample(test_sample))

             
